In [31]:
from keras import layers
import tensorflow as tf
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)


df = pd.read_csv('csv_dateien/starcraftFinalcsvs/allRunsButOnlyTerran.csv')

conditions = [
    (df['fraction'] == "Protoss"), #1
    (df['fraction'] == "Terraner"), #2
    (df['fraction'] == "Zerg"), #3
    (df['fraction'] == "T"), #2
]

values = [1, 2, 3, 2]

df["fraction"] = np.select(conditions, values)
df=df[df["total_army"]>0]
df=df[df["fraction"]==2]
df["ratio"] = df["total_workers"]/df["total_army"]

dfPrepSample = df.drop(columns=["player","winner","replay_filename", "Unnamed: 0", "total_workers", "total_army_value","fraction"])
len(dfPrepSample)


3360

In [32]:
df_sample = dfPrepSample.sample(frac=1, random_state=12)
dfTrainSample = df_sample.sample(frac=0.8)

dfTrainSample.head()

,resource_mining,total_army,supply,time,ratio
5014,5731,12.0,54.0,335,2.500000
3673,24669,30.0,78.0,613,1.400000
4389,72891,122.0,212.0,1040,0.606557
4150,6711,9.0,54.0,339,3.777778
4270,8378,14.0,62.0,401,2.500000


In [33]:
dfTestSample = df_sample.drop(dfTrainSample.index)

dfTestSample.head()

,resource_mining,total_army,supply,time,ratio
3694,15673,21.0,62.0,675,1.952381
4084,13418,47.0,110.0,583,1.000000
4621,24128,28.0,109.0,689,2.142857
1237,41604,91.0,212.0,903,0.791209
4586,19899,45.0,125.0,645,1.311111


In [34]:
dfTrainFeatures = dfTrainSample.copy()
dfTrainFeatures

,resource_mining,total_army,supply,time,ratio
5014,5731,12.0,54.0,335,2.500000
3673,24669,30.0,78.0,613,1.400000
4389,72891,122.0,212.0,1040,0.606557
4150,6711,9.0,54.0,339,3.777778
4270,8378,14.0,62.0,401,2.500000
...,...,...,...,...,...
4078,25098,112.0,205.0,831,0.598214
5094,38823,74.0,150.0,857,0.756757
3393,33677,75.0,165.0,803,0.826667
4432,10269,37.0,86.0,493,1.189189


In [35]:

dfTrainLabels = dfTrainFeatures.pop("total_army")
dfTrainFeatures = np.array(dfTrainFeatures)
dfTrainFeatures

array([[ 5731.   ,    54.   ,   335.   ,     2.5  ],
       [24669.   ,    78.   ,   613.   ,     1.4  ],
       [72891.   ,   212.   ,  1040.   ,     0.607],
       ...,
       [33677.   ,   165.   ,   803.   ,     0.827],
       [10269.   ,    86.   ,   493.   ,     1.189],
       [23317.   ,   149.   ,   702.   ,     1.327]])

In [36]:
normalize = layers.Normalization()
normalize.adapt(dfTrainFeatures)

In [41]:
sc_model = tf.keras.Sequential([layers.Dense(30, activation="relu"),layers.Dense(10, activation="relu"), layers.Dense(1)]) # letzte lösung 30/5

sc_model.compile(loss = tf.keras.losses.MeanSquaredError(),
                      optimizer = tf.keras.optimizers.Adam())

In [42]:
sc_model.fit(dfTrainFeatures, dfTrainLabels, epochs=600)

Epoch 1/600
84/84 [==============================] - 0s 576us/step - loss: 38851.0547
Epoch 2/600
84/84 [==============================] - 0s 556us/step - loss: 10531.3213
Epoch 3/600
84/84 [==============================] - 0s 559us/step - loss: 9715.8594
Epoch 4/600
84/84 [==============================] - 0s 554us/step - loss: 4848.4790
Epoch 5/600
84/84 [==============================] - 0s 550us/step - loss: 1745.6754
Epoch 6/600
84/84 [==============================] - 0s 572us/step - loss: 871.1983
Epoch 7/600
84/84 [==============================] - 0s 555us/step - loss: 1373.3798
Epoch 8/600
84/84 [==============================] - 0s 560us/step - loss: 901.1837
Epoch 9/600
84/84 [==============================] - 0s 550us/step - loss: 735.9844
Epoch 10/600
84/84 [==============================] - 0s 543us/step - loss: 859.5552
Epoch 11/600
84/84 [==============================] - 0s 558us/step - loss: 1660.7124
Epoch 12/600
84/84 [==============================] - 0s 567us/st

In [43]:
dfTestFeatures = dfTestSample.copy()

dfTestLabels = dfTestFeatures.pop("total_army")
dfTestFeatures = np.array(dfTestFeatures)
dfTestLabels.head()

3694    21.0
4084    47.0
4621    28.0
1237    91.0
4586    45.0
Name: total_army, dtype: float64

In [44]:
from sklearn.metrics import r2_score


predictedValues = sc_model.predict(dfTestFeatures)
r2_score_sc = (r2_score(dfTestLabels, predictedValues))
stri = ""
if(r2_score_sc<0):
    stri = "negative" + str(r2_score_sc)
else:
    sti = str(r2_score_sc)

print(f"The R^2 value in the test set is {stri}")

21/21 [==============================] - 0s 566us/step
The R^2 value in the test set is 


In [45]:
# optimizer with scaling and lin reg

import warnings
import gurobipy as gp
import matplotlib.pyplot as plt
from sklearn import tree
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.compose import make_column_transformer
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr

data = pd.read_csv('csv_dateien/starcraftFinalcsvs/stackedRun.csv')
data = data[1350:]


# ,player,total_workers,total_army_value,total_army,fraction,winner,replay_filename
dfOpt = dfPrepSample.drop(columns=[
                               "ratio"])

dfOpt.head


# create variables
# create Model

m = gp.Model("Balance optimizer")
gppd.set_interactive()

r = gppd.add_vars(m, pandas_obj=dfOpt,
                  name="ratio")

a = gppd.add_vars(m, pandas_obj=dfOpt,
                  name="total_army")
s = gppd.add_vars(m, pandas_obj=dfOpt,
                  name="supply")
w = m.addVar(name="workers")

m.update()
dfOpt = dfOpt.drop(columns=["total_army"])

feats = dfOpt
feats = pd.concat([r, feats], axis=1)
feats.head


m.setObjective(a.sum(), gp.GRB.MAXIMIZE) # Zielfunktion
"""
gppd.add_constrs(m, ((av - 50*w)/100), gp.GRB.GREATER_EQUAL, a) # Gleichung 2
"""
gppd.add_constrs(m, r * a, gp.GRB.EQUAL, w)
gppd.add_constrs(m, w + a, gp.GRB.GREATER_EQUAL, 0) # Gleichung 3
gppd.add_constrs(m, w + a, gp.GRB.LESS_EQUAL, s) # Gleichung 4
m.update()

pred_constr = add_predictor_constr(m, sc_model, feats, a) # Einfügen des Regressiosmodells
pred_constr.print_stats()
m.Params.NonConvex = 2
m.optimize()


outputs = pred_constr.input_values
pred_constr.input_values


solution = pd.DataFrame()

solution["ratio"] = r.gppd.X
solution["supply"] = s.gppd.X

solution
custom_palette = ["#dd8452","#4c72b0",  "#55a868"]
sns.set_theme(style="ticks")
# Show the joint distribution using kernel density estimation
g = sns.jointplot(
    data=solution, palette=custom_palette,
    x="ratio", y="supply",  # ylim=[None, 150],
    kind="kde", height=10
)

<bound method NDFrame.head of       resource_mining  total_army  supply  time
1               28154        96.0   174.0   836
6               40998       120.0   196.0   934
15              13965        45.0   117.0   583
19               9595        26.0    62.0   449
24              38030        98.0   165.0  1102
...               ...         ...     ...   ...
5891            49553       110.0   212.0  1100
5892            49464       123.0   219.0  1122
5893            26351        70.0   157.0   795
5894            24263        76.0   165.0   754
5895            22655        40.0   125.0   726

[3360 rows x 4 columns]>